<a href="https://colab.research.google.com/github/tracyxoxo/TCC-IA/blob/main/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Por se tratar de um ambiente linux, rodando o jupyter notebok, é necessário:

### 1. Instalar o google chrome
### 2. Instalar chrome driver
### 3. Instalar a biblioteca selenium

In [ ]:
# Instalação do google chrome
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install

# Instalação do chromedriver
!wget -O chromedriver_linux64.zip https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.113/linux64/chromedriver-linux64.zip
!unzip -o chromedriver_linux64.zip
!mv -f chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

# Instalação da bilbioteca para o webcrawler
!pip install -q selenium

### **WebCrawler para capturar dados historicos do INMET.**

#### **Configurado para executar o webcrawler no modo "headless" (sem interface)**

In [ ]:
# Biblioteca selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Bibliotecas para manipulação webcrawler
import time
import os
import requests

# Configurações iniciais do google chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=chrome_options)

url = "https://portal.inmet.gov.br/dadoshistoricos"
driver.get(url)
time.sleep(3)

save_folder = "/content/inmet_files"
os.makedirs(save_folder, exist_ok=True)

def get_all_article_links(driver):
    links = []
    index = 1
    while True:
        try:
            xpath = f"/html/body/div[4]/div/div/article[{index}]/a"
            element = driver.find_element("xpath", xpath)
            href = element.get_attribute("href")
            print(f"Found link #{index}: {href}")
            links.append(href)
            index += 1
        except Exception as e:
            print(f"Parou no article {index}, erro: {e}")
            break
    return links

all_links = get_all_article_links(driver)

for i, link in enumerate(all_links):
    print(f"Baixando arquivo {i+1} de {len(all_links)}: {link}")
    response = requests.get(link)
    if response.status_code == 200:
        filename = link.split("/")[-1].split("?")[0]
        path = os.path.join(save_folder, filename)
        with open(path, "wb") as f:
            f.write(response.content)
        print(f"Salvo em {path}")
    else:
        print(f"Erro ao baixar {link}: Status {response.status_code}")

driver.quit()

### **Bibliotecas da aplicação**

In [ ]:
# Baixando diretamente do repositório o arquivo utilities.py
!wget https://raw.githubusercontent.com/tracyxoxo/TCC-IA/refs/heads/dev/utilities.py

# Bibliotecas da aplicação
import pandas as pd
import matplotlib.pyplot as plt
from shutil import move
from utilities import extract_filtered_files,standard_hour,create_incorrect_data_matrix, plot_two_variables_vs_time
from datetime import time
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### **Função proveniente do arquivo utilities para filtro dos arquivos com base na torre**

In [ ]:
filtered_files = extract_filtered_files(
    files_path=r"/content/inmet_files",
    extract_path=r"/data_extracted",
    pattern='INMET_SE_SP_A705_BAURU'
)

filtered_files

#### **Processando dados dos diferentes anos disponíveis no website do INMET com base na torre selecionada (BAURU - SP) (2001 - 2025)**

In [ ]:
df_list = []

files = filtered_files

for file in files:
    df = pd.read_csv(file, encoding="ISO-8859-1", sep=";", skiprows=8, header=0)
    df = df.rename(columns={
        'DATA (YYYY-MM-DD)': 'Data',
        'HORA (UTC)' : 'Hora UTC',
        'RADIACAO GLOBAL (KJ/m²)' : 'RADIACAO GLOBAL (Kj/m²)'
    })

    df_list.append(df)
    general_df = pd.concat(df_list, ignore_index=True)

In [ ]:
general_df

### **Removendo coluna vazia**



In [ ]:
general_df.drop(['Unnamed: 19'], axis=1, inplace=True)

### **Verificando os tipos iniciais de cada coluna**

In [ ]:
general_df.dtypes

## Descrição das colunas
**PRECIPITAÇÃO TOTAL, HORÁRIO (mm)** – Quantidade total de chuva acumulada em um intervalo de uma hora, medida em milímetros (mm).    

**PRESSÃO ATMOSFÉRICA AO NÍVEL DA ESTAÇÃO, HORÁRIA (mB)** – Pressão atmosférica medida no local da estação meteorológica em milibares (mB).  

**PRESSÃO ATMOSFÉRICA NA HORA ANT. (AUT) (mB)** – Valor da pressão atmosférica registrado na hora anterior.    

**RADIAÇÃO GLOBAL (Kj/m²)** – Quantidade total de energia solar recebida por metro quadrado de superfície em uma hora, medida em kilojoules por metro quadrado (Kj/m²).    

**TEMPERATURA DO AR - BULBO SECO, HORÁRIA (°C)** – Temperatura do ar medida por um termômetro comum (sem influência da umidade), expressa em graus Celsius (°C).  

**TEMPERATURA DO PONTO DE ORVALHO (°C)** – Temperatura na qual o ar se torna saturado de umidade e ocorre a condensação, formando orvalho.  

**TEMPERATURA NA HORA ANT. (AUT) (°C)** – Temperatura do ar registrada na hora anterior.  

**TEMPERATURA ORVALHO NA HORA ANT. (AUT) (°C)** – Temperatura do ponto de orvalho registrada na hora anterior.  

**UMIDADE REL. NA HORA ANT. (AUT) (%)** – Umidade relativa do ar registrada na hora anterior.  

**UMIDADE RELATIVA DO AR, HORÁRIA (%)** – Quantidade de vapor d'água presente no ar em relação à quantidade máxima que ele pode conter a uma determinada temperatura, expressa em porcentagem.  

**VENTO, DIREÇÃO HORÁRIA (gr) (° (gr))** – Direção média do vento ao longo da última hora, medida em graus (°) em relação ao norte.  

**VENTO, RAJADA MÁXIMA (m/s)** – Maior velocidade do vento em um curto intervalo de tempo dentro da última hora, medida em metros por segundo (m/s).  

**VENTO, VELOCIDADE HORÁRIA (m/s)** – Velocidade média do vento ao longo da última hora, medida em metros por segundo (m/s).



### **Conversão do tipos de cada coluna**

In [ ]:
#general_df = backup_df

general_df['Data'] = general_df['Data'].str.replace('-','/')
general_df['Data'] = pd.to_datetime(general_df['Data'])

general_df['Hora UTC'] = general_df['Hora UTC'].apply(standard_hour)
general_df['Hora UTC'] = pd.to_datetime(general_df['Hora UTC'], format='%H%M').dt.time

general_df['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'] = general_df['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'].str.replace(',', '.')
general_df['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'] = general_df['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)'].astype(float)

general_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] = general_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].str.replace(',', '.')
general_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] = general_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].astype(float)

general_df['PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'] = general_df['PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'].str.replace(',', '.')
general_df['PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'] = general_df['PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'].astype(float)

general_df['PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)'] = general_df['PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)'].str.replace(',','.')
general_df['PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)'] = general_df['PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)'].astype(float)

general_df['PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)'] = general_df['PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)'].str.replace(',','.')
general_df['PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)'] = general_df['PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)'].astype(float)

general_df['RADIACAO GLOBAL (Kj/m²)'] = general_df['RADIACAO GLOBAL (Kj/m²)'].str.replace(',','.')
general_df['RADIACAO GLOBAL (Kj/m²)'] = general_df['RADIACAO GLOBAL (Kj/m²)'].astype(float)

general_df['TEMPERATURA DO PONTO DE ORVALHO (°C)'] = general_df['TEMPERATURA DO PONTO DE ORVALHO (°C)'].str.replace(',','.')
general_df['TEMPERATURA DO PONTO DE ORVALHO (°C)'] = general_df['TEMPERATURA DO PONTO DE ORVALHO (°C)'].astype(float)

general_df['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'].str.replace(',','.')
general_df['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)'].astype(float)

general_df['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)'].str.replace(',','.')
general_df['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)'].astype(float)

general_df['TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)'].str.replace(',','.')
general_df['TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)'].astype(float)

general_df['TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)'].str.replace(',','.')
general_df['TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)'] = general_df['TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)'].astype(float)

general_df['VENTO, RAJADA MAXIMA (m/s)'] = general_df['VENTO, DIREÇÃO HORARIA (gr) (° (gr))'].astype(float)

general_df['VENTO, VELOCIDADE HORARIA (m/s)'] = general_df['VENTO, VELOCIDADE HORARIA (m/s)'].str.replace(',','.')
general_df['VENTO, VELOCIDADE HORARIA (m/s)'] = general_df['VENTO, VELOCIDADE HORARIA (m/s)'].astype(float)

#### **Colunas convertidas para criação dos gráficos**

In [ ]:
general_df.info()

### **Analisando o por quê RADIACAO GLOBAL tem um padrão de valores nulos**

In [ ]:
radiacao_nan_df = general_df[general_df['RADIACAO GLOBAL (Kj/m²)'].isnull()]
na_por_hora = radiacao_nan_df['Hora UTC'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
na_por_hora.plot(kind='bar', color='orange')
plt.title('Frequência de Nulls por horário na radiação solar')
plt.xlabel('Horário')
plt.ylabel('Número de Nulls')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

### Removendo os registros que tem RADIACAO GLOBAL nula e hora entre 6 da manhã e 20 da noite

In [ ]:
inicio = time(6, 0)
fim = time(20, 0)

filtro = (general_df['RADIACAO GLOBAL (Kj/m²)'].isnull()) & (general_df['Hora UTC'] >= inicio) & (general_df['Hora UTC'] < fim)

general_df = general_df[~filtro]

general_df.isnull().sum()

In [ ]:
general_df['RADIACAO GLOBAL (Kj/m²)'] = general_df['RADIACAO GLOBAL (Kj/m²)'].fillna(0)

general_df.isnull().sum()

In [ ]:
general_df.info()

In [ ]:
general_df

In [ ]:
general_df.describe()

In [ ]:
general_df.plot(kind='box',figsize=(100,20), subplots=True)

### **Concatenando 'Data' com 'HORA UTC'**

In [ ]:
general_df['datetime'] = general_df.apply(
    lambda row: pd.Timestamp.combine(row['Data'],row['Hora UTC']),
    axis = 1
)

In [ ]:
general_df['datetime']

### Criando uma flag para dados incorretos

### Matriz de gráficos do tipo scatter com as variáveis definidas

In [ ]:
fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=[
        "Temperatura",
        "Pressão Atmosférica",
        "Umidade Relativa",
        "Velocidade do Vento",
        "Precipitação",
        ""
    ]
)

create_incorrect_data_matrix(general_df, 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', -50, 60, "Temperatura", row=1, col=1, fig=fig)
create_incorrect_data_matrix(general_df, 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)', 0, 2000, "Pressão atmosférica", row=1, col=2, fig=fig)
create_incorrect_data_matrix(general_df, 'UMIDADE RELATIVA DO AR, HORARIA (%)', 0, 300, "Umidade relativa do ar", row=2, col=1, fig=fig)
create_incorrect_data_matrix(general_df, 'VENTO, VELOCIDADE HORARIA (m/s)', 0, 1000, "Velocidade do vento", row=2, col=2, fig=fig)
create_incorrect_data_matrix(general_df, 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 0, 1000, "Precipitação", row=3, col=1, fig=fig)


fig.update_layout(
    height=1200,
    width=1000,
    title_text="Variáveis Meteorológicas com Anomalias",
    legend_title="Dados incorretos [Contagem]"
)

fig.show()

### Quantidade de dados incorretos (-9999)
* Temperatura x Tempo: 11%
* Pressão x Tempo: 11%
* Umidade x Tempo: 12.3%
* Vento x Tempo: 11.2%
* Precipitação x tempo: 11.2%




In [ ]:
plot_two_variables_vs_time(
    general_df,
    var1='TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    var2='UMIDADE RELATIVA DO AR, HORARIA (%)',
    var1_label='Temperatura (°C)',
    var2_label='Umidade Relativa (%)',
    title='Temperatura e Umidade ao longo do tempo'
)